In [1]:
import numpy as np
div = 10
r1 = [0.01**(2*(div-i))/div for i in range(div)]
r2 = [(i+1)*1.0/div for i in range(div-1)]
r3 = [1.0-0.1**(i)/div for i in range(div)]
rng = np.concatenate([r1,r2,r3])
print( rng)

[  1.00000000e-41   1.00000000e-37   1.00000000e-33   1.00000000e-29
   1.00000000e-25   1.00000000e-21   1.00000000e-17   1.00000000e-13
   1.00000000e-09   1.00000000e-05   1.00000000e-01   2.00000000e-01
   3.00000000e-01   4.00000000e-01   5.00000000e-01   6.00000000e-01
   7.00000000e-01   8.00000000e-01   9.00000000e-01   9.00000000e-01
   9.90000000e-01   9.99000000e-01   9.99900000e-01   9.99990000e-01
   9.99999000e-01   9.99999900e-01   9.99999990e-01   9.99999999e-01
   1.00000000e+00]


In [2]:
import numpy as np
div = 15
t_pow = 5
r1 = [(10.0**t_pow)*0.01**(2*(div-i))/div for i in range(div)]
r2 = [(10.0**t_pow)*(i+1)*1.0/div for i in range(div-1)]
r3_pow = 1.0
r3 = [((10.0**t_pow)-(0.1**r3_pow)**(i - (t_pow/r3_pow))/div) for i in range(div)]
rng = np.concatenate([r1,r2,r3])
print(rng)

[  6.66666667e-57   6.66666667e-53   6.66666667e-49   6.66666667e-45
   6.66666667e-41   6.66666667e-37   6.66666667e-33   6.66666667e-29
   6.66666667e-25   6.66666667e-21   6.66666667e-17   6.66666667e-13
   6.66666667e-09   6.66666667e-05   6.66666667e-01   6.66666667e+03
   1.33333333e+04   2.00000000e+04   2.66666667e+04   3.33333333e+04
   4.00000000e+04   4.66666667e+04   5.33333333e+04   6.00000000e+04
   6.66666667e+04   7.33333333e+04   8.00000000e+04   8.66666667e+04
   9.33333333e+04   9.33333333e+04   9.93333333e+04   9.99333333e+04
   9.99933333e+04   9.99993333e+04   9.99999333e+04   9.99999933e+04
   9.99999993e+04   9.99999999e+04   1.00000000e+05   1.00000000e+05
   1.00000000e+05   1.00000000e+05   1.00000000e+05   1.00000000e+05]


In [3]:
def test(yTarget, yPred, div=10):    
#     r1 = [0.01**(2*(div-i))/div for i in range(div)]
    r2 = [(i+1)*1.0/div for i in range(div-1)]
#     r3 = [1.0-0.1**(i)/div for i in range(div)]
#     rng = np.concatenate([r1,r2,r3])
    rng = r2
#     t_pow = 5
#     r1 = [(10.0**t_pow)*0.01**(2*(div-i))/div for i in range(div)]
#     r2 = [(10.0**t_pow)*(i+1)*1.0/div for i in range(div-1)]
#     r3_pow = 1.0
#     r3 = [((10.0**t_pow)-(0.1**r3_pow)**(i - (t_pow/r3_pow))/div) for i in range(div)]
#     rng = np.concatenate([r1,r2,r3])
    
    TP_FP_FN = np.zeros((len(rng) + div, 3))
    precision_recall = np.zeros((len(rng) + div, 2))
    
    
    
    yTarget = np.reshape(yTarget, (-1,64*64*64))
    yPred = np.reshape(yPred, (-1,64*64*64))
#         print np.sum(yTarget)
#         print np.sum(yPred)
    for prob_idx in range(len(rng)):
        print(prob_idx, len(rng))
        prob = rng[prob_idx]
#                 yTarget_t = np.where(yTarget>prob, 1.0, 0.0)
        yPred_t = np.where(yPred>prob, 1.0, 0.0)
        TP_FP_FN[prob_idx][0] += np.sum(yTarget*yPred_t)
        TP_FP_FN[prob_idx][1] += np.sum((1.0-yTarget)*yPred_t)
        TP_FP_FN[prob_idx][2] += np.sum(yTarget*(1.0-yPred_t))
    for accept_idx in range(div):
        print(accept_idx, range(div))
        accept_prob = 0.1**(accept_idx)
        accept_int = np.random.choice([0, 1], size=(len(yPred),64*64*64), p=[1.0-accept_prob, accept_prob])
#                 yTarget_t = np.greater_equal(yTarget, 1.0)
#                 yTarget_t = yTarget
        yPred_t = np.greater_equal(yPred, 1.0) * accept_int
        TP_FP_FN[len(rng)+accept_idx][0] += np.sum(yTarget*yPred_t)
        TP_FP_FN[len(rng)+accept_idx][1] += np.sum((1.0-yTarget)*yPred_t)
        TP_FP_FN[len(rng)+accept_idx][2] += np.sum(yTarget*(1.0-yPred_t))
        
    for i in range(len(precision_recall)):
        print(i, len(precision_recall))
        TP = TP_FP_FN[i][0]
        FP = TP_FP_FN[i][1]
        FN = TP_FP_FN[i][2]
        precision_recall[i][0] = TP/(TP+FP+1e-10)
        precision_recall[i][1] = TP/(TP+FN+1e-10)
    return precision_recall

In [ ]:
import numpy as np
import os
datapath = 'data/modelnet/data_AE/'
missing_pr = 0.3
# category_labels = np.load('/learn_train/'+str(missing_pr)+'_cl_label.npy')
output_images_gt = np.load(os.path.join(datapath, 'learn_train/'+str(missing_pr)+'_gt.npy'))
output_images_preds = np.load(os.path.join(datapath, 'learn_train/'+str(missing_pr)+'_pred.npy'))

pr_preds = test(output_images_gt, output_images_preds, div=20)
np.save(os.path.join(datapath, 'learn_train/'+str(missing_pr)+'_pr_preds.npy'), pr_preds)
np.savetxt('data/eval/txt/'+'modelnet_'+str(missing_pr*10)+'_AE.txt', pr_preds)

0 19
1 19
2 19
3 19
4 19
5 19
6 19
7 19
8 19
9 19
10 19
11 19
12 19
13 19
14 19
15 19
16 19
17 19
